```r
bact_data = read.csv("../datasets/CP_Phylum.csv")

bact_data_formated <- bact_data[,-1]
rownames(bact_data_formated) <- bact_data[,1]

library(reshape2)
library(seriation)
library(philentropy)

bact_data_melted = melt(bact_data)

#bact_data_t = t(bact_data)
# Simple heatmap
ggplot(bact_data_melted, aes(x=phylum, y=variable, fill=value)) + 
  geom_tile() +
  theme(
    axis.text.x = element_text(angle = 90, vjust = 1, size = 10, hjust = 1),
    axis.text.y = element_text(size=10)
  ) 
# scale_fill_manual(values = colorRampPalette(c("red", "blue", "green")))(42) +

##### Seriate Hmap Library ########
#dist_cor <- function(x) as.dist(distance(x, method = "eucledian", use.row.names = TRUE))
dist_cor <- function(x) dist(x, method = "euclidean")

hmap(bact_data_formated, method="GW", distfun = dist, col=greenred(100))
hmap(t(bact_data_formated), method="OLO", distfun = dist, col=greenred(100))

hmap(bact_data_formated, method="MDS_angle", distfun = dist_cor, showdist = "both", col=greenred(100)) 

#### Lattice plots ###
# library(lattice)
# levelplot(as.matrix(bact_data_formated), col.regions=rainbow(100)) #cm.colors(100) topo.colors(100) rainbow(100) #heat.colors(100) #terrain.colors(100)
###

#### D3 Heatmap apply Library ###########

library(d3heatmap)
d3heatmap(bact_data_formated, scale = "column")

d3heatmap(bact_data_formated, scale = "column", dendrogram = "row", color = scales::col_quantile("Blues", NULL, 5))

# use k_row or k_col for k mean cluster highlight in dendrograms of axis
d3heatmap(bact_data_formated, scale = "column", colors = "Blues", dendrogram = "row", k_row = 3)

# custom distance and clustering function
dist_cor <- function(x) dist(x, method = "euclidean")
hclust_fun <- function(x) hclust(x, method = "average")

## Reorder the y axis ## Reorder function args - dendrogram, weights
library(seriation)
reorder_fun <- function(d, w) reorder(d, dist(bact_data_formated, method = "euclidean"), method = "OLO") 

# Reorder function without using seration
#reorder_fun <- function(d, w) reorder(d, w, agglo.FUN = mea)  

d3heatmap(bact_data_formated, scale = "column", distfun = dist_cor, hclustfun = hclust_fun, reorderfun = reorder_fun)

##### Try plotly for interective heatmap plots
#library(plotly)
#plot_ly(
#  x = bact_data_melted$phylum, 
#  y = bact_data_melted$variable,
#  z = bact_data_melted$value, 
#  type = "heatmap"
#)
#####

#### Heatmap apply Library ###########
######################################
#### BEST for Dendrogram heatmaps ####

library(heatmaply)

heatmaply(
  bact_data_formated,
  seriate = "OLO"
)

# use k means to color dendrograms k_col and k_row

heatmaply(
  bact_data_formated,
  seriate = "OLO",
  xlab = "Strain",
  ylab = "Taxa",
  k_col = 3,
  k_row = 3,
  
  fontsize_col = 10,
  fontsize_row = 10,
  
  plot_method= "plotly",
  hclust_method = "average", #upgma average linkage
)

##### ***** Use library dendextend to play with dendrogram
# library(dendextend)

##### Heatmaps with correlation tests ######

bact_cor_mat = cor(bact_data_formated) # Get corrrelation between bacteria taxa # Using Trasposed data_frame

## Custom function for calculating p values
cor.test.p <- function(x){
  FUN <- function(x, y) cor.test(x, y)[["p.value"]]
  z <- outer(
    colnames(x), 
    colnames(x), 
    Vectorize(function(i,j) FUN(x[,i], x[,j]))
  )
  dimnames(z) <- list(colnames(x), colnames(x))
  z
}

cor_p_vals = cor.test.p(bact_data_formated) # Using Trasposed data_frame

#### Correlation matrix plot ######
heatmaply_cor(
  bact_cor_mat,
  node_type = "scatter",
  point_size_mat = -log10(cor_p_vals), 
  point_size_name = "-log10(p-value)",
  label_names = c("x", "y", "Correlation")
)



```